In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tensornetworks_pytorch.TNModels import PosMPS, Born
print(torch.__version__)

1.8.0.dev20201128


Use some data

In [2]:
import pickle
for dataset in [#'biofam',
    'flare','lymphography','spect','tumor','votes']:
    with open('datasets/'+dataset, 'rb') as f:
            a=pickle.load(f)
    X=a[0].astype(int)
    print(dataset)
    print("\tdata shape:", X.shape)
    print(f"\trange of X values: {X.min()} -- {X.max()}")

def load_dataset(dataset):
    with open('datasets/'+dataset, 'rb') as f:
            a=pickle.load(f)
    X=a[0]
    X=X.astype(int)

    print("data shape:", X.shape)
    print(f"range of X values: {X.min()} -- {X.max()}")
    print(f"setting d={X.max()+1}")
    d = X.max()+1
    return X, d

flare
	data shape: (1065, 13)
	range of X values: 0 -- 7
lymphography
	data shape: (148, 19)
	range of X values: 0 -- 7
spect
	data shape: (187, 23)
	range of X values: 0 -- 1
tumor
	data shape: (339, 17)
	range of X values: 0 -- 3
votes
	data shape: (435, 17)
	range of X values: 0 -- 2


In [12]:
X,d = load_dataset('flare')

data shape: (1065, 13)
range of X values: 0 -- 7
setting d=8


In [15]:
D = 4
mps = PosMPS(dataset=X, D=D, d=d, homogeneous=False, verbose=True)
rBorn = Born(dataset=X, D=D, d=d, dtype=torch.float, homogeneous=False, verbose=True) 
cBorn = Born(dataset=X, D=D, d=d, dtype=torch.cfloat, verbose=True)
models = (mps, rBorn, cBorn)
for model in models:
    print(model.core.shape, model.name)

nonhomogeneous init
nonhomogeneous init
torch.Size([13, 8, 4, 4]) Positive MPS
torch.Size([13, 8, 4, 4]) Born model torch.float32
torch.Size([8, 4, 4]) Born model torch.complex64


In [17]:
from tqdm.notebook import tqdm
batchsize=20
trainloader = DataLoader(X, batch_size=batchsize, shuffle=True)
model = cBorn
optimizer = torch.optim.SGD(model.parameters(), lr=0.2, momentum=0.5)

max_epochs = 20
print("Training...")
av_batch_loss_running = -1000
for epoch in tqdm(range(max_epochs)):
    print("epoch", epoch)
    batch_loss = []
    for batch_idx, batch in enumerate(trainloader):
        if torch.isnan(model.core).any():
            print("NANS!")
            break
        model.zero_grad()
        neglogprob = 0
        for x in batch:
            out = model(x)
            neglogprob -= out
#             print("out",out)
        loss = neglogprob / len(batch)
#         print("core befor\n",model.core[0][0])
#         if batch_idx>0:
#             print("\tgrad:",model.core.grad[0,0])
        loss.backward()
#         print("backward.")
#         print("\tgrad:",model.core.grad[0,0])
        optimizer.step()
#         print("optimize\n",model.core[0][0])
#         print("\tgrad:",model.core.grad[0,0])
        with torch.no_grad():
            batch_loss.append(loss.item())
            n=10
            if batch_idx % n == 0: # print every nth batch loss
                print("\tbatch", batch_idx, "loss", loss.item())
    av_batch_loss = torch.Tensor(batch_loss).mean().item()
    print("\tavg batch_loss", av_batch_loss)
    if abs(av_batch_loss_running - av_batch_loss) < .0001:
        print("Early stopping")
        break
    av_batch_loss_running = av_batch_loss
print('Finished training. Last av loss = ', av_batch_loss)

Training...


epoch 0
	batch 0 loss 27.538593292236328
NANS!
	avg batch_loss 27.538593292236328
epoch 1
NANS!
	avg batch_loss nan
epoch 2
NANS!
	avg batch_loss nan
epoch 3
NANS!
	avg batch_loss nan
epoch 4
NANS!
	avg batch_loss nan
epoch 5
NANS!
	avg batch_loss nan
epoch 6
NANS!
	avg batch_loss nan
epoch 7
NANS!
	avg batch_loss nan
epoch 8
NANS!
	avg batch_loss nan
epoch 9
NANS!
	avg batch_loss nan
epoch 10
NANS!
	avg batch_loss nan
epoch 11
NANS!
	avg batch_loss nan
epoch 12
NANS!
	avg batch_loss nan
epoch 13
NANS!
	avg batch_loss nan
epoch 14
NANS!
	avg batch_loss nan
epoch 15
NANS!
	avg batch_loss nan
epoch 16
NANS!
	avg batch_loss nan
epoch 17
NANS!
	avg batch_loss nan
epoch 18
NANS!
	avg batch_loss nan
epoch 19
NANS!
	avg batch_loss nan

Finished training. Last av loss =  nan


In [10]:
model.core.grad[0,0]

tensor([0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j])

In [98]:
real = torch.tensor([1, 2], dtype=torch.float32)
imag = torch.tensor([3, 4], dtype=torch.float32)
z = torch.complex(real, imag)
z.requires_grad=True
b = (2.7j*z + 3.4*z + 18)
L = b.mean().abs().square()
L.backward()
print(L.grad_fn)
print(z.grad.conj())

tensor([89.4750-17.3750j, 89.4750-17.3750j])


In [66]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
L = (a * a).sum()
print(L.grad_fn, a.grad_fn)
L.backward()
print(a.grad)

False
True
<SumBackward0 object at 0x7f80ed99aee0> None
tensor([[-0.1552,  4.3674],
        [ 1.3441,  1.1090]])


In [ ]:
pip install tensorboard
pip install tensorboardX